In [1]:
import random
import numpy as np
import tensorflow as tf
import os

# Set seeds
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)
os.environ['TF_DETERMINISTIC_OPS'] = '1'

In [ ]:
import pandas as pd
import numpy as np
import os

meta = pd.read_csv('Train_Test_Split.csv')
meta = meta[meta['label'].isin(['AD', 'Healthy'])]

train_meta = meta[meta['split'] == 'train']
test_meta = meta[meta['split'] == 'test']

def load_and_segment(subject_id, data_dir='Data_sampled_128HZ', segment_len=1024):
    file_path = os.path.join(data_dir, f"{subject_id}_data.npy")
    data = np.load(file_path)
    _, time_steps = data.shape
    num_segments = time_steps // segment_len
    if num_segments == 0:
        return np.empty((0, 19, segment_len))
    data = data[:, :num_segments * segment_len]
    segments = data.reshape(19, num_segments, segment_len).transpose(1, 0, 2)
    return segments

def process_data(meta_df, data_dir='Data_sampled_128HZ'):
    X = []
    y = []
    label_map = {'AD': 1, 'Healthy': 0}
    for _, row in meta_df.iterrows():
        segments = load_and_segment(row['subject_id'], data_dir)
        if segments.shape[0] == 0:
            continue
        X.append(segments)
        label = label_map[row['label']]
        one_hot = np.eye(2)[label]
        y.extend([one_hot] * segments.shape[0])
    X = np.concatenate(X, axis=0)
    y = np.array(y)
    return X, y
X_train, y_train = process_data(train_meta)
X_test, y_test = process_data(test_meta)
X_train = (X_train * 1e6) - np.mean(X_train * 1e6, axis=2, keepdims=True)
X_test = (X_test * 1e6) - np.mean(X_test * 1e6, axis=2, keepdims=True)

In [3]:
#BiLSTM Start to End
top_channels = [14, 2, 0, 18, 4]  # comes from tSNE
X_train_selected = X_train[:, top_channels, :].transpose(0, 2, 1)  # (samples, 1024, 5)
X_test_selected = X_test[:, top_channels, :].transpose(0, 2, 1)    # (samples, 1024, 5)

from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Bidirectional, BatchNormalization, ReLU
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

def create_multi_channel_bilstm(input_shape):
    model = Sequential([
        Bidirectional(LSTM(128, return_sequences=True), input_shape=input_shape),
        BatchNormalization(),
        ReLU(),
        Dropout(0.4),
        Bidirectional(LSTM(70)),
        BatchNormalization(),
        ReLU(),
        Dropout(0.4),
        Dense(50, activation='relu'),
        Dense(2, activation='softmax')
    ])

    model.compile(
        optimizer=Adam(0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy', 'AUC']
    )
    return model

model = create_multi_channel_bilstm((1024, 5))

callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6, verbose=1)
]

history = model.fit(
    X_train_selected, y_train,
    validation_data=(X_test_selected, y_test),
    epochs=50,
    batch_size=32,
    callbacks=callbacks,
    verbose=1
)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
164/164 ━━━━━━━━━━━━━━━━━━━━ 34s 157ms/step - AUC: 0.5134 - accuracy: 0.5113 - loss: 0.8034 - val_AUC: 0.5517 - val_accuracy: 0.5418 - val_loss: 0.6905 - learning_rate: 0.0010
Epoch 2/50
164/164 ━━━━━━━━━━━━━━━━━━━━ 37s 154ms/step - AUC: 0.5448 - accuracy: 0.5351 - loss: 0.7055 - val_AUC: 0.5562 - val_accuracy: 0.5401 - val_loss: 0.6975 - learning_rate: 0.0010
Epoch 3/50
164/164 ━━━━━━━━━━━━━━━━━━━━ 41s 152ms/step - AUC: 0.5681 - accuracy: 0.5464 - loss: 0.6925 - val_AUC: 0.5840 - val_accuracy: 0.5660 - val_loss: 0.6849 - learning_rate: 0.0010
Epoch 4/50
164/164 ━━━━━━━━━━━━━━━━━━━━ 41s 152ms/step - AUC: 0.6053 - accuracy: 0.5761 - loss: 0.6798 - val_AUC: 0.6177 - val_accuracy: 0.5884 - val_loss: 0.6730 - learning_rate: 0.0010
Epoch 5/50
164/164 ━━━━━━━━━━━━━━━━━━━━ 41s 154ms/step - AUC: 0.6343 - accuracy: 0.6011 - loss: 0.6660 - val_AUC: 0.6138 - val_accuracy: 0.5867 - val_loss: 0.6997 - learning_rate: 0.0010
Epoch 6/50
164/164 ━━━━━━━━━━━━━━━━━━━━ 41s 153ms/step - AUC: 0.6

In [4]:
from sklearn.metrics import classification_report, roc_auc_score, average_precision_score

y_pred = model.predict(X_test_selected)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)

print("ROC AUC:", roc_auc_score(y_true, y_pred[:, 1]))
print("Average Precision:", average_precision_score(y_true, y_pred[:, 1]))
print(classification_report(y_true, y_pred_classes))

37/37 ━━━━━━━━━━━━━━━━━━━━ 3s 66ms/step
ROC AUC: 0.8797597651569614
Average Precision: 0.8895647804041216
              precision    recall  f1-score   support

           0       0.84      0.71      0.77       535
           1       0.78      0.89      0.83       624

    accuracy                           0.81      1159
   macro avg       0.81      0.80      0.80      1159
weighted avg       0.81      0.81      0.80      1159



In [ ]:
output_folder = "Models"
os.makedirs(output_folder,exist_ok=True)
model_path = os.path.join(output_folder,"Final_Bilstm_model.keras")
model.save(model_path)